### 社会力模型实现避障到达目标

1、目标对个体吸引力<br>
2、不同个体间相互排斥力<br>
3、障碍物对个体排斥力

In [ ]:
# 导入需要用到的库
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
import json
import plotly.graph_objects as go
from pyproj import Proj,transform

#### 定义基础函数

In [48]:
# 定义三维向量，表示位置坐标及高度
def vec3(x, y, z):
    input_proj = Proj(init = 'epsg:4326')   #wgs84坐标
    output_proj = Proj(init = 'epsg:3857')  #web墨卡托投影坐标
    x,y = transform(input_proj,output_proj,x,y)
    return np.array([x, y, z], dtype=np.float64)

# 计算向量长度
def length(vec):
    return np.linalg.norm(vec)

# 计算单位向量向量
def normalize(vec):
    if length(vec) == 0:
        return vec
    return vec / length(vec)

# 限制向量的长度
def truncate(vec, max_value):
    if length(vec) > max_value:
        return normalize(vec) * max_value
    return vec

def limit_turn(steering, max_turn_rate):
    if length(steering) > max_turn_rate:
        return normalize(steering) * max_turn_rate
    return steering

#### 可视化

In [67]:
def plot_trajectories(drones, target_position):
    fig = go.Figure()

    # 绘制每架无人机的轨迹
    for i, drone in enumerate(drones):
        path = np.array(drone.path)
        
        # 绘制无人机轨迹线
        fig.add_trace(go.Scatter3d(
            x=path[:, 0], y=path[:, 1], z=path[:, 2],
            mode='lines',
            name=f'UAV {i+1}',
            line=dict(width=4)
        ))

        # 标记无人机最终位置
        fig.add_trace(go.Scatter3d(
            x=[path[-1, 0]], y=[path[-1, 1]], z=[path[-1, 2]],
            mode='markers',
            name=f'UAV {i+1} Final Position',
            marker=dict(size=6)
        ))

    # 绘制目标点
    input_proj = Proj(init = 'epsg:4326')   #wgs84坐标
    output_proj = Proj(init = 'epsg:3857')  #web墨卡托投影坐标

    x1,y1 = transform(output_proj,input_proj,target_position[0], target_position[1])
    fig.add_trace(go.Scatter3d(
        x=[x1], y=[y1], z=[target_position[2]],
        mode='markers',
        name='Target',
        marker=dict(color='red', size=8, symbol='x')
    ))

    # 图形标题和轴标签
    fig.update_layout(
        title='UAV Trajectories',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        ),
        showlegend=True
    )

    fig.show()


#### 主要功能类

In [ ]:
# 定义无人机类
class UAV:
    def __init__(self, position, velocity=None, mass=400.0, max_speed=10.0, max_turn_rate=20, radius=0.2, repulsion_distance=1.0):
        self.position = position
        self.velocity = velocity if velocity is not None else vec3(0, 0, 0)
        self.mass = mass
        self.max_speed = max_speed
        self.max_turn_rate = max_turn_rate
        self.acceleration = vec3(0, 0, 0)
        self.radius = radius
        self.repulsion_distance = repulsion_distance
        self.path = []
        self.reached_target = False    # 是否到达目标点

    # 计算加速度
    def apply_force(self, force):
        self.acceleration = force / self.mass
    
    # 更新位置和速度信息
    def update(self, time_elapsed, target_position, others):
        if self.reached_target:
            self.velocity = vec3(0, 0, 0)
            return

        steering_force = self.seek(target_position)    # 目标吸引力
        repulsion_force = self.avoid_collision(others) # 个体排斥力
        total_force = steering_force + repulsion_force
        self.apply_force(total_force)

        # 更新速度和位置
        self.velocity += self.acceleration * time_elapsed
        self.velocity = truncate(self.velocity, self.max_speed)
        new_position = self.position + self.velocity * time_elapsed

        # 检查是否已到达目标
        if 0 < length(target_position - new_position) < 10.0:
            self.velocity = vec3(0, 0, 0)  # 速度清零
            self.reached_target = True

        # 更新位置并记录路径
        self.position = new_position
        input_proj = Proj(init='epsg:3857')  # 墨卡托投影
        output_proj = Proj(init='epsg:4326')  # WGS84地理坐标
        x,y = transform(input_proj,output_proj,self.position[0], self.position[1])
        self.path.append([x, y, self.position[2]])


    # 目标吸引力
    def seek(self, target_position, deceleration=0.1):
        desired_velocity = target_position - self.position
        distance_to_target = length(desired_velocity)

        if distance_to_target > 0:
            speed = distance_to_target / deceleration
            speed = min(speed, self.max_speed)
            desired_velocity = desired_velocity * (speed / distance_to_target)
            return limit_turn(desired_velocity - self.velocity, self.max_turn_rate)

        return vec3(0, 0, 0)
    
    # 无人机之间排斥力的计算
    def avoid_collision(self, others):
        min_repulsion_distance = self.repulsion_distance
        total_repulsion = vec3(0, 0, 0)
        
        for other in others:
            if other is self:
                continue
            
            direction_to_other = other.position - self.position
            distanceSQ = length(direction_to_other)
            
            if distanceSQ < (self.repulsion_distance if not self.reached_target else min_repulsion_distance) ** 2:
                distance = np.sqrt(distanceSQ)
                normalized_direction = normalize(direction_to_other)
                # 使用较小的排斥力系数，如果无人机已到达目标
                repulsion_strength = (100 if self.reached_target else 1000) / (distance ** 2 + 1)
                total_repulsion += -normalized_direction * repulsion_strength

        return total_repulsion


# 检测两架无人机之间的距离
def check_distances(drones, min_distance=10.0):
    for i in range(len(drones)):
        for j in range(i + 1, len(drones)):
            distance = np.linalg.norm(drones[i].position - drones[j].position)
            if distance < min_distance:
                print(f"Warning: Distance between UAV {i+1} and UAV {j+1} is {distance:.2f} meters, which is below the minimum distance of {min_distance} meters.")

# 模拟
def simulate(drones, target_position, time_elapsed, steps):
    for step in range(steps):
        all_reached = True
        for drone in drones:
            drone.update(time_elapsed, target_position, drones)
            if not drone.reached_target:
                all_reached = False
        
        check_distances(drones)
        if all_reached:
            print("All drones have reached their target.")
            break


In [86]:
drones = [
    UAV(position=vec3(121.4738, 31.2305, 0)),
    UAV(position=vec3(121.4739, 31.2304, 0)),
    UAV(position=vec3(121.4741, 31.2304, 0)),
    UAV(position=vec3(121.4739, 31.2305, 0))
]

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

C:\Users\a\AppData\Local\Temp\ipykernel_33160\3372815867.py:5: FutureWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authorit

In [70]:
drones[0].path

[]

In [71]:
# 输出初始状态
for i, drone in enumerate(drones):
    print(f"UAV {i+1} Initial Position: {drone.position}, Velocity: {drone.velocity}")

UAV 1 Initial Position: [13522401.56072395  3662720.2755213         0.        ], Velocity: [0. 0. 0.]
UAV 2 Initial Position: [13522412.69267303  3662707.2570504         0.        ], Velocity: [0. 0. 0.]
UAV 3 Initial Position: [13522434.95657119  3662707.2570504         0.        ], Velocity: [0. 0. 0.]
UAV 4 Initial Position: [13522412.69267303  3662720.2755213         0.        ], Velocity: [0. 0. 0.]


In [87]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore", FutureWarning)

# 设置目标位置和其他模拟参数
target_position = vec3(121.4738, 31.2305, 50)
time_elapsed = 1
steps = 500

# 开始模拟
simulate(drones, target_position, time_elapsed=time_elapsed, steps=steps)

# 可视化运动轨迹
plot_trajectories(drones, target_position)


d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

C:\Users\a\AppData\Local\Temp\ipykernel_33160\3372815867.py:5: FutureWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authorit

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

C:\Users\a\AppData\Local\Temp\ipykernel_33160\375621894.py:44: FutureWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authorit

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

C:\Users\a\AppData\Local\Temp\ipykernel_33160\375621894.py:44: FutureWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authorit

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

C:\Users\a\AppData\Local\Temp\ipykernel_33160\375621894.py:44: FutureWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authorit

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

C:\Users\a\AppData\Local\Temp\ipykernel_33160\3372815867.py:5: FutureWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authorit

...

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

C:\Users\a\AppData\Local\Temp\ipykernel_33160\3372815867.py:5: FutureWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authorit

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

C:\Users\a\AppData\Local\Temp\ipykernel_33160\3372815867.py:5: FutureWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authorit

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

C:\Users\a\AppData\Local\Temp\ipykernel_33160\375621894.py:44: FutureWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authorit

All drones have reached their target.


d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

C:\Users\a\AppData\Local\Temp\ipykernel_33160\3372815867.py:5: FutureWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

d:\Environment\Conda\envs\daily_env\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authorit

#### 完整版

In [ ]:
# 定义三维向量，表示位置坐标及高度
def vec3(x, y, z):
    return np.array([x, y, z], dtype=np.float64)

# 计算向量长度
def length(vec):
    return np.linalg.norm(vec)

# 计算单位向量向量
def normalize(vec):
    if length(vec) == 0:
        return vec
    return vec / length(vec)

# 限制向量的长度
def truncate(vec, max_value):
    if length(vec) > max_value:
        return normalize(vec) * max_value
    return vec

def limit_turn(steering, max_turn_rate):
    if length(steering) > max_turn_rate:
        return normalize(steering) * max_turn_rate
    return steering
# 定义无人机类
class UAV:
    # 质量400g，最大速度10m/s，最大转向速度20rad/s，无人机半径0.2m，排斥距离10m
    def __init__(self, position, velocity=None, mass=400.0, max_speed=10.0, max_turn_rate=20, radius=0.2, repulsion_distance=10.0):
        
        self.position = position
        self.velocity = velocity if velocity is not None else vec3(0, 0, 0)    # 若不输入，默认无人机初始速度为0
        self.mass = mass
        self.max_speed = max_speed
        self.max_turn_rate = max_turn_rate
        self.acceleration = vec3(0, 0, 0)
        self.radius = radius
        self.repulsion_distance = repulsion_distance
        self.path = [self.position]

        self.reached_target = False    # 是否到达目标点


    # 计算加速度
    def apply_force(self, force):
        # a = f/m
        self.acceleration = force / self.mass
    
    # 更新位置和速度信息
    def update(self, time_elapsed, target_position, others):
        
        if self.reached_target:
            self.velocity = vec3(0, 0, 0)
            return

        # 计算合力
        steering_force = self.seek(target_position)    # 目标吸引力
        repulsion_force = self.avoid_collision(others)    # 个体排斥力
        total_force = steering_force + repulsion_force    # 未添加权重系数
        self.apply_force(total_force)

        # 更新速度和位置
        self.velocity += self.acceleration * time_elapsed    # v = v + at
        self.velocity = truncate(self.velocity, self.max_speed)
        new_position = self.position + self.velocity * time_elapsed    # 新的位置

        # 检查是否已到达目标
        if length(target_position - new_position) < 50.0:  
            
            self.velocity = vec3(0, 0, 0)  # 速度清零
            self.reached_target = True    # 标记为已到达目标


        # 更新位置并记录路径
        self.position = new_position
        self.path.append(self.position.copy())

    # 目标吸引力
    def seek(self, target_position, deceleration=0.3):
        
        # 无人机与目标的初始距离
        desired_velocity = target_position - self.position
        distance_to_target = length(desired_velocity)

        if distance_to_target > 50:

            speed = distance_to_target / deceleration
            speed = min(speed, self.max_speed)
            desired_velocity = desired_velocity * (speed / distance_to_target)
            
            return limit_turn(desired_velocity - self.velocity, self.max_turn_rate)

        return vec3(0, 0, 0)
    
    # 个体之间排斥力
    def avoid_collision(self, others):
        
        if self.reached_target:
            return vec3(0, 0, 0)

        total_repulsion = vec3(0, 0, 0)    
        for other in others:
            if other is self:
                continue

            # 计算方向和距离平方
            direction_to_other = other.position - self.position
            distanceSQ = length(direction_to_other)
            
            if distanceSQ > self.repulsion_distance ** 2:
                continue

            # 距离在10米以内时应用排斥力
            distance = np.sqrt(distanceSQ)
            normalized_direction = normalize(direction_to_other)

            # 使用距离平方反比的排斥力计算公式
            repulsion_strength = 1000 / (distance ** 2 + 1)  # 调整强度系数，+1 避免极端情况
            total_repulsion += -normalized_direction * repulsion_strength

        return total_repulsion




# 检测两架无人机之间的距离
def check_distances(drones, min_distance=10.0):
    for i in range(len(drones)):
        for j in range(i + 1, len(drones)):
            distance = np.linalg.norm(drones[i].position - drones[j].position)
            if distance < min_distance:
                print(f"Warning: Distance between UAV {i+1} and UAV {j+1} is {distance:.2f} meters, which is below the minimum distance of {min_distance} meters.")

# 模拟
def simulate(drones, target_position, time_elapsed, steps):
    for step in range(steps):
        all_reached = True
        for drone in drones:
            drone.update(time_elapsed, target_position, drones)
            if not drone.reached_target:
                all_reached = False
            # 调试输出
            #print(f"Step {step}, UAV Position: {drone.position}")
        
        check_distances(drones)
        if all_reached:
            print("All drones have reached their target.")
            break


#### 连接前端版

In [83]:
import json
import numpy as np
import time
from pyproj import Proj,transform

# 定义三维向量，表示位置坐标及高度
# 输入坐标转换为投影坐标

def vec3(lon, lat, z):
    input_proj = Proj(init = 'epsg:4326')   #wgs84坐标
    output_proj = Proj(init = 'epsg:3857')  #web墨卡托投影坐标
    x,y = transform(input_proj,output_proj,lon,lat)
    return np.array([x, y, z], dtype=np.float64)

# 计算向量长度
def length(vec):
    return np.linalg.norm(vec)

# 计算单位向量向量
def normalize(vec):
    if length(vec) == 0:
        return vec
    return vec / length(vec)

# 限制向量的长度
def truncate(vec, max_value):
    if length(vec) > max_value:
        return normalize(vec) * max_value
    return vec

def limit_turn(steering, max_turn_rate):
    if length(steering) > max_turn_rate:
        return normalize(steering) * max_turn_rate
    return steering

# 定义无人机类
class UAV:

    # 使用标准单位，无人机重量400g，最大速度20m/s，最大转向速度20rad/s，半径0.2m，排斥半径10m
    # 未添加：最大可倾斜角35°
    def __init__(self, position, velocity=None, mass=400.0, max_speed=20.0, max_turn_rate=20, radius=0.2, repulsion_distance=10.0):
        self.position = position
        self.velocity = velocity if velocity is not None else vec3(0, 0, 0)
        self.mass = mass
        self.max_speed = max_speed
        self.max_turn_rate = max_turn_rate
        self.acceleration = vec3(0, 0, 0)
        self.radius = radius
        self.repulsion_distance = repulsion_distance
        self.path = [self.position]
        self.reached_target = False

    # 计算加速度
    def apply_force(self, force):
        # a = f/m
        self.acceleration = force / self.mass
    
    # 更新位置和速度信息
    def update(self, time_elapsed, target_position, others):
        
        if self.reached_target:
            self.velocity = vec3(0, 0, 0)
            return

        # 计算合力
        steering_force = self.seek(target_position)    # 目标吸引力
        repulsion_force = self.avoid_collision(others)    # 个体排斥力
        total_force = steering_force + repulsion_force    # 未添加权重系数
        self.apply_force(total_force)

        # 更新速度和位置
        self.velocity += self.acceleration * time_elapsed    # v = v + at
        self.velocity = truncate(self.velocity, self.max_speed)
        new_position = self.position + self.velocity * time_elapsed    # 新的位置

        # 检查是否已到达目标
        if length(target_position - new_position) < 50.0:  
            
            self.velocity = vec3(0, 0, 0)  # 速度清零
            self.reached_target = True    # 标记为已到达目标


        # 更新位置并记录路径
        self.position = new_position
        self.path.append(self.position.copy())

    # 目标吸引力
    def seek(self, target_position, deceleration=0.3):
        
        # 无人机与目标的初始距离
        desired_velocity = target_position - self.position
        distance_to_target = length(desired_velocity)

        if distance_to_target > 50:

            speed = distance_to_target / deceleration
            speed = min(speed, self.max_speed)
            desired_velocity = desired_velocity * (speed / distance_to_target)
            
            return limit_turn(desired_velocity - self.velocity, self.max_turn_rate)

        return vec3(0, 0, 0)
    
    # 个体之间排斥力
    def avoid_collision(self, others):
        
        if self.reached_target:
            return vec3(0, 0, 0)

        total_repulsion = vec3(0, 0, 0)    
        for other in others:
            if other is self:
                continue

            # 计算方向和距离平方
            direction_to_other = other.position - self.position
            distanceSQ = length(direction_to_other)
            
            if distanceSQ > self.repulsion_distance ** 2:
                continue

            # 距离在10米以内时应用排斥力
            distance = np.sqrt(distanceSQ)
            normalized_direction = normalize(direction_to_other)

            # 使用距离平方反比的排斥力计算公式
            repulsion_strength = 1000 / (distance ** 2 + 1)  # 调整强度系数，+1 避免极端情况
            total_repulsion += -normalized_direction * repulsion_strength

        return total_repulsion

# 检测两架无人机之间的距离
def check_distances(drones, min_distance=10.0):
    for i in range(len(drones)):
        for j in range(i + 1, len(drones)):
            distance = np.linalg.norm(drones[i].position - drones[j].position)
            if distance < min_distance:
                print(f"Warning: Distance between UAV {i+1} and UAV {j+1} is {distance:.2f} meters, which is below the minimum distance of {min_distance} meters.")

# 读取JSON文件并初始化无人机
def load_drones_from_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    drones = []
    targets = []

    for entry in data:
        pos = vec3(entry['geometry']['x'], entry['geometry']['y'], entry['geometry']['z'])
        target = vec3(entry['target']['targetx'], entry['target']['targety'], entry['target']['targetz'])
        drones.append(UAV(position=pos))
        targets.append(target)

    return drones, targets



def simulate(input_json_path, time_step=1, steps=10):
    # 读取 JSON 文件
    with open(input_json_path, 'r') as file:
        data = json.load(file)
    
    drones = []
    all_results = []  # 用于存储所有无人机的最终输出数据

    for drone_data in data:
        # 初始化无人机属性
        position = vec3(drone_data["geometry"]["x"], drone_data["geometry"]["y"], drone_data["geometry"]["z"])
        velocity = vec3(0, 0, 0)  # 初始化速度
        max_speed = drone_data["baseinfo"]["maxspeed"]
        max_turn_rate = drone_data["baseinfo"]["maxturnrate"]

        # 创建无人机实例并初始化输出结构
        drone = UAV(
            position=position,
            velocity=velocity,
            max_speed=max_speed,
            max_turn_rate=max_turn_rate,
            radius=drone_data["baseinfo"]["Dradius"],
            repulsion_distance=drone_data["baseinfo"]["CAradius"]
        )
        drones.append(drone)

        # 初始化输出数据结构
        all_results.append({
            "id": drone_data["id"],
            "time_interval": time_step,
            "statusinfo": {
                "speeds": [],  # 记录每一步的速度
                "ifarrival": 0  # 最终的到达状态
            },
            "path": {
                "nx": [],  # 记录每一步的x坐标
                "ny": [],  # 记录每一步的y坐标
                "nz": []   # 记录每一步的z坐标
            }
        })

    target_positions = [vec3(d["target"]["targetx"], d["target"]["targety"], d["target"]["targetz"]) for d in data]
    input_proj = Proj(init='epsg:3857')  # 墨卡托投影
    output_proj = Proj(init='epsg:4326')  # WGS84地理坐标

    for step in range(steps):
        for i, drone in enumerate(drones):
            drone.update(time_step, target_positions[i], drones)

            # 转换投影坐标到地理坐标
            lon, lat = transform(input_proj, output_proj, drone.position[0], drone.position[1])
            altitude = drone.position[2]

            # 记录每一步的数据
            all_results[i]["path"]["nx"].append(lon)
            all_results[i]["path"]["ny"].append(lat)
            all_results[i]["path"]["nz"].append(altitude)
            all_results[i]["statusinfo"]["speeds"].append(np.linalg.norm(drone.velocity))

            # 如果无人机到达目标，将到达标志设为1
            if drone.reached_target:
                all_results[i]["statusinfo"]["ifarrival"] = 1

        # 暂停以模拟每秒生成数据
        time.sleep(time_step)

    # 保存所有无人机的数据到一个 JSON 文件
    with open("output_all_steps.json", 'w') as output_file:
        json.dump(all_results, output_file, indent=2)

simulate("drones.json", time_step=1, steps=10)


d:\APPs\Anaconda\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

d:\APPs\Anaconda\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

C:\Users\a\AppData\Local\Temp\ipykernel_4308\638916321.py:12: FutureWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

d:\APPs\Anaconda\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is t

#### 地理坐标转换

In [1]:
import pyproj
from shapely.geometry import Point

In [2]:
# 定义两个地理点
point1 = Point(50.67, 4.62)
point2 = Point(51.67, 4.64)

# 使用WGS84椭球体进行距离计算
geod = pyproj.Geod(ellps='WGS84')

# 计算两个点之间的距离和方位角
angle1, angle2, distance = geod.inv(point1.x, point1.y, point2.x, point2.y)

print(f"方位角: {angle1}, 逆方位角: {angle2}, 距离: {distance / 1000:.2f} 公里")


方位角: 88.8177981917099, 逆方位角: -91.10147893212144, 距离: 110.98 公里


In [3]:
from geopy.distance import geodesic
 
# 参数为两个元组，每个元组包含经度和纬度
coord_1 = (39.917978, 116.396288) # 北京天安门坐标
coord_2 = (31.230416, 121.473701) # 上海市区坐标
distance = geodesic(coord_1, coord_2).m # 距离结果单位为千米
 
print("距离为：{:.2f}米".format(distance)) # 输出距离结果

距离为：1067639.42米


In [ ]:
from pyproj import Proj,transform

input_proj = Proj(init = 'epsg:4326')   #wgs84坐标
output_proj = Proj(init = 'epsg:3857')  #web墨卡托投影坐标

lon,lat = 118,32
x,y = transform(input_proj,output_proj,lon,lat)

print(x,y)

13135699.913606282 3763310.627144653


d:\APPs\Anaconda\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
d:\APPs\Anaconda\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\a\AppData\Local\Temp\ipykernel_28876\465443729.py:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x,y = transform(input_proj,output_

In [7]:
lon1,lat1 = 116.396288,39.917978
lon2,lat2 = 121.473701,31.230416

x1,y1 = transform(input_proj,output_proj,lon1,lat1)
x2,y2 = transform(input_proj,output_proj,lon2,lat2)

print(x1,y1)
print(x2,y2)

12957175.510387218 4854030.214980274
13522390.540094366 3662709.3400048204


C:\Users\a\AppData\Local\Temp\ipykernel_28876\615266012.py:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x1,y1 = transform(input_proj,output_proj,lon1,lat1)
C:\Users\a\AppData\Local\Temp\ipykernel_28876\615266012.py:5: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2,y2 = transform(input_proj,output_proj,lon2,lat2)


In [9]:
(x1-x2)**2 + (y1-y2)**2

1738713456959.1345

In [10]:
1067639.42**2

1139853931137.9363